In [1]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from skimage.color import rgb2gray, rgba2rgb
from matplotlib.image import imread
#!pip install -q keras
from keras.models import Sequential
# from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, ReLU, Dropout
from tensorflow.keras.utils import to_categorical

class_names = ['flute', 'viola', 'cello', 'oboe', 'trumpet', 'saxophone']
number_of_classes = len(class_names)

instruments = {'flute':1, 'viola':2, 'cello':3, 'oboe':4, 'trumpet':5, 'saxophone':6}

files = glob.glob('BasicsMusicalInstrumClassifi/audio/london_phill_dataset_multi/spectrograms2/*.png')
np.random.shuffle(files)
data, labels = [], []

loaded = 0

print('Number of samples {0}'.format(len(files)))

for file in files:
  img, name = imread(file), os.path.basename(file).split('_')[0]#file.split('/')[-1].split('_')[0]
  img=rgba2rgb(img)
  data.append(rgb2gray(img))
  labels.append(instruments[name] - 1)

  loaded += 1
  if loaded % 200 == 0:
    print(loaded)

  if loaded == 5000:
    break

data, labels = np.array(data), np.array(labels)

s_60p, s_20p = int(len(data) * 0.6), int(len(data) * 0.2)

X_train, y_train = data[:s_60p], labels[:s_60p] #60%
X_validation, y_validation = data[s_60p: s_60p + s_20p], labels[s_60p: s_60p + s_20p]  #20%
X_test, y_test = data[s_60p + s_20p:], labels[s_60p + s_20p:] #20%

print(X_train.shape, X_test.shape)


In [ ]:
print(X_validation.shape, y_validation.shape)

In [ ]:
print (X_train.shape[0])

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 540, 1080,1)
X_test = X_test.reshape(X_test.shape[0], 540, 1080, 1)
X_validation = X_validation.reshape(X_validation.shape[0], 540, 1080, 1)

print('Dataset has been loaded succesfully.')
print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)

print(to_categorical(y_train).shape)
print(to_categorical(y_validation).shape)
print(X_train.shape[1:])

epochs = 30

model = Sequential([
        Conv2D(32, (3, 3), input_shape=(540, 1080, 1)),
        Activation('relu'),
        Conv2D(32, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(64, (3,3), padding='same'),
        Activation('relu'),
        Conv2D(64, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Conv2D(128, (3, 3), padding='same'),
        Activation('relu'),
        Conv2D(128, (3, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(3, 3)),
        Dropout(0.25),

        Flatten(),
        Dense(128),
        Activation('relu'),
        Dropout(0.5),
        Dense(number_of_classes),
        Activation('softmax')
    ])


In [ ]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())



In [ ]:
print('Fitting started...')
history = model.fit(X_train, to_categorical(y_train), epochs=20, validation_data=(X_validation, to_categorical(y_validation)), batch_size=4)

In [ ]:
#model accuracy
print(history.history)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

#model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(X_test, to_categorical(y_test), verbose=1)
print(scores)